## Importing NIH Dataset (ZIP Format)

In [1]:
# Import relevant packages
import numpy as np
import os
from shutil import copyfile
from zipfile import ZipFile

# Download NIH dataset zip file
!wget -nc ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip

# Extract images if not already extracted
ROOT_DIR = os.path.join("/", "content")
if not os.path.isdir("cell_images"):
    print("Extracting images...")
    with ZipFile(os.path.join("cell_images.zip"), "r") as zipObj:
        zipObj.extractall()
    print("Done!")

--2020-02-18 06:31:39--  ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip
           => ‘cell_images.zip’
Resolving lhcftp.nlm.nih.gov (lhcftp.nlm.nih.gov)... 130.14.55.35, 2607:f220:41e:7055::35
Connecting to lhcftp.nlm.nih.gov (lhcftp.nlm.nih.gov)|130.14.55.35|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /Open-Access-Datasets/Malaria ... done.
==> SIZE cell_images.zip ... 353452851
==> PASV ... done.    ==> RETR cell_images.zip ... done.
Length: 353452851 (337M) (unauthoritative)

cell_images.zip     100%[===================>] 337.08M  14.9MB/s    in 28s     

2020-02-18 06:32:09 (12.0 MB/s) - ‘cell_images.zip’ saved [353452851]

Extracting images...
Done!


## Unzip Images, Resize, and Store in NumPy Arrays

In [2]:
# Install and import relevant packages
import numpy as np
import os
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6 libxrender1
import cv2
from PIL import Image

# Create new folders to save rescaled images
if not os.path.isdir("RescaledSet"):
    os.mkdir("RescaledSet")
if not os.path.isdir("RescaledSet/Parasitized"):
    os.mkdir("RescaledSet/Parasitized")
if not os.path.isdir("RescaledSet/Uninfected"):
    os.mkdir("RescaledSet/Uninfected")

# Generate list of parasitized file names
ParasitizedFiles = os.listdir("cell_images/Parasitized/")
UninfectedFiles = os.listdir("cell_images/Uninfected/")

# Remove Thumb.db files
while 'Thumbs.db' in ParasitizedFiles: ParasitizedFiles.remove('Thumbs.db')   
while 'Thumbs.db' in UninfectedFiles: UninfectedFiles.remove('Thumbs.db')  

# Pre-allocate memory space for images
Parasitized = np.empty([13779,128,128,3])
Uninfected = np.empty([13779,128,128,3])

# Resize and load parasitized images
for i in range(13779):
    TempImage = cv2.imread('cell_images/Parasitized/'+ParasitizedFiles[i])
    ResizedImage = cv2.resize(TempImage, dsize=(128,128))
    Parasitized[i,:,:,:] = ResizedImage

# Resize and load uninfected images
for i in range(13779):
    TempImage = cv2.imread('cell_images/Uninfected/'+UninfectedFiles[i])
    ResizedImage = cv2.resize(TempImage, dsize=(128,128))
    Uninfected[i,:,:,:] = ResizedImage
    
print('Uninfected Dataset size is:',np.shape(Uninfected))
print('Parasitized Dataset size is:',np.shape(Parasitized))

     |████████████████████████████████| 28.2 MB 32 kB/s  eta 0:00:01
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease          
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease                
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done 33m
Building dependency tree       
Reading state information... Done
28 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Don

## Generate Cross-Validation Indices for Training and Testing Sets

In [4]:
# Generate dataset labels
ParasitizedLabels = np.repeat([[0,1]], 13779, axis=0)
UninfectedLabels = np.repeat([[1,0]], 13779, axis=0)
Labels = np.concatenate((ParasitizedLabels,UninfectedLabels), axis=0)

# Generate image dataset
Dataset = np.concatenate((Parasitized, Uninfected), axis=0)

# Generate 5-fold cross-validation groups
CVIndices = np.random.permutation(Dataset.shape[0])
Index1, Index2, Index3, Index4, Index5 = CVIndices[:5512], CVIndices[5512:11024], CVIndices[11024:16536], CVIndices[16536:22048], CVIndices[22048:]

## Train Model and Save Results as CSV Files

In [ ]:
# Import relevant packages for neural network training
import sys
import csv
if 'tensorflow' in sys.modules == False:
    %tensorflow_version 2.x
    import tensorflow as tf
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

!pip install scikit-learn
import sklearn
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

# Import relevant neural network architecture packages 
from keras.applications.vgg16 import VGG16

# Specify layers to freeze
FreezeLayers = [19,15,11,7,4,0]

for j in FreezeLayers:
    
    # Create empty lists to store results
    TrainLoss = []
    TrainAcc = []
    TestLoss = []
    TestAcc = []
    All_FPR = []
    All_TPR = []
    All_thresholds = []
    All_AUC = []

    for i in range(5):

        # Create the appropriate training and testing sets
        if i == 0:
            TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:]), axis=0)
            TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index3,:], Labels[Index4,:]), axis=0)
            TestImages = Dataset[Index5,:]
            TestLabels = Labels[Index5,:]
        elif i == 1:
            TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index3,:],Dataset[Index5,:]), axis=0)
            TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index3,:], Labels[Index5,:]), axis=0)
            TestImages = Dataset[Index4,:]
            TestLabels = Labels[Index4,:]
        elif i == 2:
            TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index2,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
            TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index2,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
            TestImages = Dataset[Index3,:]
            TestLabels = Labels[Index3,:]
        elif i == 3:
            TrainImages = np.concatenate((Dataset[Index1,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
            TrainLabels = np.concatenate((Labels[Index1,:], Labels[Index3,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
            TestImages = Dataset[Index2,:]
            TestLabels = Labels[Index2,:]
        else:
            TrainImages = np.concatenate((Dataset[Index2,:],Dataset[Index3,:],Dataset[Index4,:],Dataset[Index5,:]), axis=0)
            TrainLabels = np.concatenate((Labels[Index2,:], Labels[Index3,:], Labels[Index4,:], Labels[Index5,:]), axis=0)
            TestImages = Dataset[Index1,:]
            TestLabels = Labels[Index1,:]

        base_model = VGG16(weights = "imagenet", include_top=False, input_shape = (128,128,3))

        for layer in base_model.layers[:j]:
            layer.trainable=False
        for k,layer in enumerate(base_model.layers):
            print(k,layer.name,layer.trainable)

        x = base_model.output
        x = Flatten()(x)
        x = Dense(1024, activation="relu")(x)
        x = Dropout(0.5)(x)
        x = Dense(1024, activation="relu")(x)
        x = Dropout(0.5)(x)
        predictions = Dense(2, activation="softmax")(x)
        model = Model(input = base_model.input, output = predictions)
        adam = optimizers.Adam(lr=0.000001, beta_1=0.9, beta_2=0.999, amsgrad=False)
        model.compile(loss = "categorical_crossentropy", optimizer = adam, metrics=["accuracy"])

        # Train model and evaluate performance
        print('We are now training cross-validation set #',i+1)
        Results = model.fit(TrainImages, TrainLabels, epochs=75, batch_size=64, validation_data=(TestImages,TestLabels), validation_freq=1)

        # Display and store performance results
        Results.history['loss'] = [round(k, 4) for k in Results.history['loss']]
        Results.history['accuracy'] = [round(k, 4) for k in Results.history['accuracy']]
        Results.history['val_loss'] = [round(k, 4) for k in Results.history['val_loss']]
        Results.history['val_accuracy'] = [round(k, 4) for k in Results.history['val_accuracy']]

        print('Training Loss:',Results.history['loss'])
        print('Training Accuracy:',Results.history['accuracy'])
        print('Validation Loss:',Results.history['val_loss'])
        print('Validation Accuracy:',Results.history['val_accuracy'])

        TrainLoss.append(Results.history['loss'])
        TrainAcc.append(Results.history['accuracy'])
        TestLoss.append(Results.history['val_loss'])
        TestAcc.append(Results.history['val_accuracy'])
        print('')


        # Predict values for test set
        Probabilities = model.predict(TestImages)

        # Calculate data for ROC curve
        FPR, TPR, thresholds = roc_curve(TestLabels[:,1], Probabilities[:,1])
        All_FPR.append(FPR)
        All_TPR.append(TPR)
        All_thresholds.append(thresholds)

        # Save and export as CSV files
        with open(str(j)+"Freeze_TrainLoss.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(TrainLoss)
        with open(str(j)+"Freeze_TrainAcc.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(TrainAcc)
        with open(str(j)+"Freeze__TestLoss.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(TestLoss)
        with open(str(j)+"Freeze__TestAcc.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(TestAcc)
        with open(str(j)+"Freeze__FPR.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(All_FPR)
        with open(str(j)+"Freeze__TPR.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(All_TPR)
        with open(str(j)+"Freeze__Thresholds.csv", "w") as f:
            writer = csv.writer(f)
            writer.writerows(All_thresholds)

0 input_2 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 True
8 block3_conv2 True
9 block3_conv3 True
10 block3_pool True
11 block4_conv1 True
12 block4_conv2 True
13 block4_conv3 True
14 block4_pool True
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
We are now training cross-validation set # 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Train on 22048 samples, validate on 5510 samples
Epoch 1/75
22048/22048 [==============================] - 28s 1ms/step - loss: 2.9410 - accuracy: 0.5930 - val_loss: 0.3870 - val_accuracy: 0.8240
Epoch 2/75
22048/22048 [==============================] - 28s 1ms/step - loss: 0.7514 - accuracy: 0.7268 - val_loss: 0.2923 - val_accuracy: 0.8868
Epoch 3/75
22048/22048 [==============================] - 28s 1ms/step - loss: 0.4771 - accuracy: 0.8081 - val_loss: 0.2482 - val_accuracy: 0.9118
Epoch 4/75
22048/22048 [==============================] - 28s 1ms/step - loss: 0.3651 - accuracy: 0.8583 - val_loss: 0.2141 - val_accuracy: 0.9290
Epoch 5/75
22048/22048 [==============================] - 28s 1ms/step - loss: 0.2939 - accuracy: 0.8874 - val_loss: 0.1809 - val_accuracy: 0.9385
Epoch 6/75
22048/22048 [==============================] - 28s 1ms/step - loss: 0.2476 - accuracy: 0.9068 - val_loss: 0.1603 - val_accuracy: 0.9456
Epoch 7/75
22048/22048 [==============================] - 28s 1ms/ste

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Train on 22046 samples, validate on 5512 samples
Epoch 1/75
22046/22046 [==============================] - 29s 1ms/step - loss: 2.6489 - accuracy: 0.6163 - val_loss: 0.4173 - val_accuracy: 0.8113
Epoch 2/75
22046/22046 [==============================] - 28s 1ms/step - loss: 0.7606 - accuracy: 0.7188 - val_loss: 0.3188 - val_accuracy: 0.8623
Epoch 3/75
22046/22046 [==============================] - 28s 1ms/step - loss: 0.4761 - accuracy: 0.7984 - val_loss: 0.2637 - val_accuracy: 0.8980
Epoch 4/75
22046/22046 [==============================] - 28s 1ms/step - loss: 0.3593 - accuracy: 0.8477 - val_loss: 0.2173 - val_accuracy: 0.9211
Epoch 5/75
22046/22046 [==============================] - 28s 1ms/step - loss: 0.2853 - accuracy: 0.8869 - val_loss: 0.1871 - val_accuracy: 0.9369
Epoch 6/75
22046/22046 [==============================] - 28s 1ms/step - loss: 0.2392 - accuracy: 0.9082 - val_loss: 0.1631 - val_accuracy: 0.9465
Epoch 7/75
22046/22046 [==============================] - 28s 1ms/ste

...

Train on 22046 samples, validate on 5512 samples
Epoch 1/75
22046/22046 [==============================] - 39s 2ms/step - loss: 2.4132 - accuracy: 0.6329 - val_loss: 0.3320 - val_accuracy: 0.8521
Epoch 2/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.6442 - accuracy: 0.7644 - val_loss: 0.1930 - val_accuracy: 0.9334
Epoch 3/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.3716 - accuracy: 0.8618 - val_loss: 0.1576 - val_accuracy: 0.9497
Epoch 4/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.2785 - accuracy: 0.9023 - val_loss: 0.1461 - val_accuracy: 0.9559
Epoch 5/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.2292 - accuracy: 0.9217 - val_loss: 0.1402 - val_accuracy: 0.9579
Epoch 6/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.2009 - accuracy: 0.9332 - val_loss: 0.1353 - val_accuracy: 0.9603
Epoch 7/75
22046/22046 [==============================] - 38s 2ms/ste

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Train on 22046 samples, validate on 5512 samples
Epoch 1/75
22046/22046 [==============================] - 40s 2ms/step - loss: 2.5319 - accuracy: 0.6453 - val_loss: 0.3377 - val_accuracy: 0.8619
Epoch 2/75
22046/22046 [==============================] - 39s 2ms/step - loss: 0.6403 - accuracy: 0.7757 - val_loss: 0.2051 - val_accuracy: 0.9218
Epoch 3/75
22046/22046 [==============================] - 39s 2ms/step - loss: 0.3785 - accuracy: 0.8541 - val_loss: 0.1664 - val_accuracy: 0.9421
Epoch 4/75
22046/22046 [==============================] - 39s 2ms/step - loss: 0.2798 - accuracy: 0.8976 - val_loss: 0.1522 - val_accuracy: 0.9470
Epoch 5/75
22046/22046 [==============================] - 39s 2ms/step - loss: 0.2292 - accuracy: 0.9210 - val_loss: 0.1467 - val_accuracy: 0.9490
Epoch 6/75
22046/22046 [==============================] - 39s 2ms/step - loss: 0.1982 - accuracy: 0.9332 - val_loss: 0.1440 - val_accuracy: 0.9514
Epoch 7/75
22046/22046 [==============================] - 39s 2ms/ste

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Train on 22046 samples, validate on 5512 samples
Epoch 1/75
22046/22046 [==============================] - 39s 2ms/step - loss: 2.6565 - accuracy: 0.5969 - val_loss: 0.4135 - val_accuracy: 0.8066
Epoch 2/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.7165 - accuracy: 0.7102 - val_loss: 0.2718 - val_accuracy: 0.8953
Epoch 3/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.4422 - accuracy: 0.8175 - val_loss: 0.1800 - val_accuracy: 0.9396
Epoch 4/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.3162 - accuracy: 0.8784 - val_loss: 0.1489 - val_accuracy: 0.9526
Epoch 5/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.2503 - accuracy: 0.9101 - val_loss: 0.1381 - val_accuracy: 0.9554
Epoch 6/75
22046/22046 [==============================] - 38s 2ms/step - loss: 0.2132 - accuracy: 0.9266 - val_loss: 0.1354 - val_accuracy: 0.9575
Epoch 7/75
22046/22046 [==============================] - 38s 2ms/ste

## Examine RAM Requirements

In [ ]:
# Examine RAM Usage
import sys
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)